In [9]:
"""
ESNET TESTBED EXPERIMENTS
"""

import os
import json
import numpy as np
from pathlib import Path

# rootdir = "/Users/eashan22/Desktop/Internship 2021/bbrv2/Brian's Project/experiment2"
rootdir = "/Users/eashan22/Desktop/experiment2"

def traverse_less_than_30(path):
    fileList = []
    c = 0
    for root, directories, files in os.walk(path):
        for file in files:
            # if file.endswith("json"):
            if file.endswith("json") and float( file.split(":")[3].split(".json")[0].split("ms")[0] )<30: # for filess where RTT is less than 30 
                fileList.append(os.path.join(root,file))
                c+=1
    print(f"Total files: {c}")
    return fileList

def traverse_greater_than_30(path):
    fileList = []
    c = 0
    for root, directories, files in os.walk(path):
        for file in files:
            # if file.endswith("json"):
            if file.endswith("json") and float( file.split(":")[3].split(".json")[0].split("ms")[0] )>30: # for filess where RTT is greater than 30 
                fileList.append(os.path.join(root,file))
                c+=1
    print(f"Total files: {c}")
    return fileList

paths = [
        (#"0.1-loss.v1/pscheduler_bbr2_p1",
        # "0.001-loss/pscheduler_bbr2_p16",
        # "0.1-loss.v1/pscheduler_bbr2_p16",
        "0.01-loss.v1/pscheduler_bbr2_p1",
        "0.01-loss.v1/pscheduler_bbr2_p16",
        "no-loss/pscheduler_bbr2_p1",
        "no-loss/pscheduler_bbr2_p16",
        "no-loss/pscheduler_bbr2_p16_v2",),

        (#"0.1-loss/pscheduler_cubic_p16",
        # "0.1-loss.v1/pscheduler_cubic_p1",   
        # "0.1-loss.v1/pscheduler_cubic_p16",
        # "0.001-loss/pscheduler_cubic_p16",
        "0.01-loss.v1/pscheduler_cubic_p1",  
        "0.01-loss.v1/pscheduler_cubic_p16",
        "no-loss/pscheduler_cubic_p1",
        "no-loss/pscheduler_cubic_p16",
        "no-loss/pscheduler_cubic_p16_v2"),

        (#"0.001-loss/pscheduler_both_p16",
        # "0.1-loss.v1/pscheduler_both_p1",
        # "0.1-loss.v1/pscheduler_both_p16",
        # "0.1-loss.v2/pscheduler_both_p16", 
        # "0.1-loss.v2/pscheduler_both_p16_bbr1",
        "0.01-loss.v1/pscheduler_both_p1",
        "0.01-loss.v1/pscheduler_both_p16",
        "no-loss/pscheduler_both_p1",
        "no-loss/pscheduler_both_p16",
        "no-loss/pscheduler_both_p16_v2")
        ]

In [10]:
# =================================================================================
# TRAVERSE LESS THAN 30 RTT
# =================================================================================

print("\n")
for q1 in paths[0]:
    print(f"============================ {q1} ============================")
    pscheduler_bbr2  = os.path.join(rootdir, q1)
    filenames = traverse_less_than_30(pscheduler_bbr2)

    data_seg = []
    tput_bbr2_p1 = []
    key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

    for i,f in enumerate(filenames):
        try:
            path = Path(f)
            data = [json.loads(line) for line in open(f, 'r')]
            
            bbr2_data_seg, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0
            for j,d in enumerate(data):
                if "interval" in d.keys() and j==0:
                    start_time = data[j]['interval']['time'] # Start time of the test
                elif key2 in data[j].keys():
                    end_time = data[j-1]['interval']['time'] # End time of the test
                    mss = data[j-1]['interval']['mss'] # maximum segment size

                    bbr2_data_seg = data[j][key2]
                    data_seg.append( bbr2_data_seg )

                    throughput = (bbr2_data_seg*mss*8)/(end_time-start_time)/1e9
                    tput_bbr2_p1.append( throughput )
        
        except Exception as e:
            print(e)

    print("Throughput (P1)")
    print(f"BBRv2 - Mean: {np.mean(tput_bbr2_p1):.5f}  |  Coef. of Variance: {(np.std(tput_bbr2_p1)/np.mean(tput_bbr2_p1)):.5f}")
    # print(f"BBRv2 - Mean: {np.mean(tput_bbr2_p1):.5f}  |  Std. Dev.: {np.std(tput_bbr2_p1):.5f}  |  Coef. of Variance: {(np.std(tput_bbr2_p1)/np.mean(tput_bbr2_p1)):.5f}  |  Variance: {np.var(tput_bbr2_p1):.5f}")
    # print("Data Segment (P1)")
    # print(f"BBRv2 - Mean: {np.mean(data_seg):.5f}  |  Std. Dev.: {np.std(data_seg):.5f}  |  Coef. of Variance: {(np.std(data_seg)/np.mean(data_seg)):.5f}  |  Variance: {np.var(data_seg):.5f}")
    print("")

for q2 in paths[1]:
    print(f"============================ {q2} ============================")
    pscheduler_cubic = os.path.join(rootdir, q2)
    filenames = traverse_less_than_30(pscheduler_cubic)

    data_seg = []
    tput_p1_cubic = []
    key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

    for i,f in enumerate(filenames):
        try:
            path = Path(f)
            # The MongoDB JSON dump has one object per line, so this works for me.
            data = [json.loads(line) for line in open(f, 'r')]
            
            cubic_data_seg, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0
            for j,d in enumerate(data):
                if "interval" in d.keys() and j==0:
                    start_time = data[j]['interval']['time'] # Start time of the test
                elif key1 in data[j].keys():
                    end_time = data[j-1]['interval']['time'] # End time of the test
                    mss = data[j-1]['interval']['mss'] # maximum segment size

                    cubic_data_seg = data[j][key1]
                    data_seg.append( cubic_data_seg )

                    throughput = (cubic_data_seg*mss*8)/(end_time-start_time)/1e9
                    tput_p1_cubic.append( throughput )
        
        except Exception as e:
            print(e)

    print("Throughput (P1)")
    print(f"CUBIC - Mean: {np.mean(tput_p1_cubic):.5f}  |  Coef. of Variance: {(np.std(tput_p1_cubic)/np.mean(tput_p1_cubic)):.5f}")
    # print(f"CUBIC - Mean: {np.mean(tput_p1_cubic):.5f}  |  Std. Dev.: {np.std(tput_p1_cubic):.5f}  |  Coef. of Variance: {(np.std(tput_p1_cubic)/np.mean(tput_p1_cubic)):.5f}  |  Variance: {np.var(tput_p1_cubic):.5f}")
    # print("Data Segment (P1)")
    # print(f"CUBIC - Mean: {np.mean(data_seg):.5f}  |  Std. Dev.: {np.std(data_seg):.5f}  |  Coef. of Variance: {(np.std(data_seg)/np.mean(data_seg)):.5f}  |  Variance: {np.var(data_seg):.5f}")
    print("")

for q3 in paths[2]:
    print(f"============================ {q3} ============================")
    pscheduler_both = os.path.join(rootdir, q3)
    filenames = traverse_less_than_30(pscheduler_both)

    data_seg_sum_cubic, data_seg_sum_bbr2 = [], []
    tput_cubic_p16, tput_bbr2_p16 = [], []
    key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

    for i,f in enumerate(filenames):
        try:
            path = Path(f)
            data = [json.loads(line) for line in open(f, 'r')]
            
            throughput_cubic, throughput_bbr2, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
            for j,d in zip(range(len(data)),data):
                try:
                    if "interval" in d.keys() and j==0:
                        start_time = d['interval']['time'] # Start time of the test
                    if "streams" in d.keys():
                        end_time = data[j-1]['interval']['time'] # End time of the test
                        mss = data[j-1]['interval']['mss'] # maximum segment size
                        
                        cubic_data_seg_list, bbr2_data_seg_list = [], []
                        for j in range(len(d['streams'])):
                            if "cubic" in d['streams'][j]['cc']:
                                cubic_data_seg_list.append(d['streams'][j]['data_segs'])
                            elif "bbr2" in d['streams'][j]['cc']:
                                bbr2_data_seg_list.append(d['streams'][j]['data_segs'])
                        
                        data_seg_sum_cubic.append( sum(cubic_data_seg_list) )
                        throughput_cubic = (sum(cubic_data_seg_list)*mss*8)/(end_time-start_time)/1e9
                        tput_cubic_p16.append( throughput_cubic )

                        data_seg_sum_bbr2.append( sum(bbr2_data_seg_list) )
                        throughput_bbr2 = (sum(bbr2_data_seg_list)*mss*8)/(end_time-start_time)/1e9
                        tput_bbr2_p16.append( throughput_bbr2 )

                except Exception as e:
                    print(e)

        except Exception as e:
            print(e)

    print("Throughput (P16)")
    print(f"BBRv2 - Mean: {np.mean(tput_bbr2_p16):.5f}  |  Coef. of Variance: {(np.std(tput_bbr2_p16)/np.mean(tput_bbr2_p16)):.5f}")
    print(f"CUBIC - Mean: {np.mean(tput_cubic_p16):.5f}  |  Coef. of Variance: {(np.std(tput_cubic_p16)/np.mean(tput_cubic_p16)):.5f}")
    # print(f"BBRv2 - Mean: {np.mean(tput_bbr2_p16):.5f}  |  Std. Dev.: {np.std(tput_bbr2_p16):.5f}  |  Coef. of Variance: {(np.std(tput_bbr2_p16)/np.mean(tput_bbr2_p16)):.5f}  |  Variance: {np.var(tput_bbr2_p16):.5f}")
    # print(f"CUBIC - Mean: {np.mean(tput_cubic_p16):.5f}  |  Std. Dev.: {np.std(tput_cubic_p16):.5f}  |  Coef. of Variance: {(np.std(tput_cubic_p16)/np.mean(tput_cubic_p16)):.5f}  |  Variance: {np.var(tput_cubic_p16):.5f}")

    # print("Data Segment (P16)")
    # print(f"BBRv2 - Mean: {np.mean(data_seg_sum_bbr2):.5f}  |  Std. Dev.: {np.std(data_seg_sum_bbr2):.5f}  |  Coef. of Variance: {(np.std(data_seg_sum_bbr2)/np.mean(data_seg_sum_bbr2)):.5f}  |  Variance: {np.var(data_seg_sum_bbr2):.5f}")
    # print(f"CUBIC - Mean: {np.mean(data_seg_sum_cubic):.5f}  |  Std. Dev.: {np.std(data_seg_sum_cubic):.5f}  |  Coef. of Variance: {(np.std(data_seg_sum_cubic)/np.mean(data_seg_sum_cubic)):.5f}  |  Variance: {np.var(data_seg_sum_cubic):.5f}")
    print(50*"=")


print(50*"*")
print(50*"*")



============================ 0.01-loss.v1/pscheduler_bbr2_p1 ============================
-- ss:10.201.1.2:4:20.0ms.json
20.0
-- ss:10.201.1.2:4:4.0ms.json
4.0
-- ss:10.201.1.2:3:10.0ms.json
10.0
-- ss:10.201.1.2:1:20.0ms.json
20.0
-- ss:10.201.1.2:1:4.0ms.json
4.0
-- ss:10.201.1.2:2:50.0ms.json
50.0
-- ss:10.201.1.2:1:50.0ms.json
50.0
-- ss:10.201.1.2:2:20.0ms.json
20.0
-- ss:10.201.1.2:3:4.0ms.json
4.0
-- ss:10.201.1.2:5:10.0ms.json
10.0
-- ss:10.201.1.2:4:50.0ms.json
50.0
-- ss:10.201.1.2:3:50.0ms.json
50.0
-- ss:10.201.1.2:2:10.0ms.json
10.0
-- ss:10.201.1.2:5:20.0ms.json
20.0
-- ss:10.201.1.2:5:4.0ms.json
4.0
-- ss:10.201.1.2:2:4.0ms.json
4.0
-- ss:10.201.1.2:5:50.0ms.json
50.0
-- ss:10.201.1.2:4:10.0ms.json
10.0
-- ss:10.201.1.2:1:10.0ms.json
10.0
-- ss:10.201.1.2:3:20.0ms.json
20.0
Total files: 15
Throughput (P1)
BBRv2 - Mean: 2.34768  |  Coef. of Variance: 0.00171

============================ 0.01-loss.v1/pscheduler_bbr2_p16 ============================
-- ss:10.201.1.2:4:20

IndexError: list index out of range

In [ ]:
# =================================================================================
# TRAVERSE GREATER THAN 30 RTT
# =================================================================================

print("\n")
for q1 in paths[0]:
    print(f"============================ {q1} ============================")
    pscheduler_bbr2  = os.path.join(rootdir, q1)
    filenames = traverse_greater_than_30(pscheduler_bbr2)

    data_seg = []
    tput_bbr2_p1 = []
    key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

    for i,f in enumerate(filenames):
        try:
            path = Path(f)
            data = [json.loads(line) for line in open(f, 'r')]
            
            bbr2_data_seg, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0
            for j,d in enumerate(data):
                if "interval" in d.keys() and j==0:
                    start_time = data[j]['interval']['time'] # Start time of the test
                elif key2 in data[j].keys():
                    end_time = data[j-1]['interval']['time'] # End time of the test
                    mss = data[j-1]['interval']['mss'] # maximum segment size

                    bbr2_data_seg = data[j][key2]
                    data_seg.append( bbr2_data_seg )

                    throughput = (bbr2_data_seg*mss*8)/(end_time-start_time)/1e9
                    tput_bbr2_p1.append( throughput )
        
        except Exception as e:
            print(e)

    print("Throughput (P1)")
    print(f"BBRv2 - Mean: {np.mean(tput_bbr2_p1):.5f}  |  Coef. of Variance: {(np.std(tput_bbr2_p1)/np.mean(tput_bbr2_p1)):.5f}")
    # print(f"BBRv2 - Mean: {np.mean(tput_bbr2_p1):.5f}  |  Std. Dev.: {np.std(tput_bbr2_p1):.5f}  |  Coef. of Variance: {(np.std(tput_bbr2_p1)/np.mean(tput_bbr2_p1)):.5f}  |  Variance: {np.var(tput_bbr2_p1):.5f}")
    # print("Data Segment (P1)")
    # print(f"BBRv2 - Mean: {np.mean(data_seg):.5f}  |  Std. Dev.: {np.std(data_seg):.5f}  |  Coef. of Variance: {(np.std(data_seg)/np.mean(data_seg)):.5f}  |  Variance: {np.var(data_seg):.5f}")
    print("")

for q2 in paths[1]:
    print(f"============================ {q2} ============================")
    pscheduler_cubic = os.path.join(rootdir, q2)
    filenames = traverse_greater_than_30(pscheduler_cubic)

    data_seg = []
    tput_p1_cubic = []
    key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

    for i,f in enumerate(filenames):
        try:
            path = Path(f)
            # The MongoDB JSON dump has one object per line, so this works for me.
            data = [json.loads(line) for line in open(f, 'r')]
            
            cubic_data_seg, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0
            for j,d in enumerate(data):
                if "interval" in d.keys() and j==0:
                    start_time = data[j]['interval']['time'] # Start time of the test
                elif key1 in data[j].keys():
                    end_time = data[j-1]['interval']['time'] # End time of the test
                    mss = data[j-1]['interval']['mss'] # maximum segment size

                    cubic_data_seg = data[j][key1]
                    data_seg.append( cubic_data_seg )

                    throughput = (cubic_data_seg*mss*8)/(end_time-start_time)/1e9
                    tput_p1_cubic.append( throughput )
        
        except Exception as e:
            print(e)

    print("Throughput (P1)")
    print(f"CUBIC - Mean: {np.mean(tput_p1_cubic):.5f}  |  Coef. of Variance: {(np.std(tput_p1_cubic)/np.mean(tput_p1_cubic)):.5f}")
    # print(f"CUBIC - Mean: {np.mean(tput_p1_cubic):.5f}  |  Std. Dev.: {np.std(tput_p1_cubic):.5f}  |  Coef. of Variance: {(np.std(tput_p1_cubic)/np.mean(tput_p1_cubic)):.5f}  |  Variance: {np.var(tput_p1_cubic):.5f}")
    # print("Data Segment (P1)")
    # print(f"CUBIC - Mean: {np.mean(data_seg):.5f}  |  Std. Dev.: {np.std(data_seg):.5f}  |  Coef. of Variance: {(np.std(data_seg)/np.mean(data_seg)):.5f}  |  Variance: {np.var(data_seg):.5f}")
    print("")

for q3 in paths[2]:
    print(f"============================ {q3} ============================")
    pscheduler_both = os.path.join(rootdir, q3)
    filenames = traverse_greater_than_30(pscheduler_both)

    data_seg_sum_cubic, data_seg_sum_bbr2 = [], []
    tput_cubic_p16, tput_bbr2_p16 = [], []
    key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

    for i,f in enumerate(filenames):
        try:
            path = Path(f)
            data = [json.loads(line) for line in open(f, 'r')]
            
            throughput_cubic, throughput_bbr2, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
            for j,d in zip(range(len(data)),data):
                try:
                    if "interval" in d.keys() and j==0:
                        start_time = d['interval']['time'] # Start time of the test
                    if "streams" in d.keys():
                        end_time = data[j-1]['interval']['time'] # End time of the test
                        mss = data[j-1]['interval']['mss'] # maximum segment size
                        
                        cubic_data_seg_list, bbr2_data_seg_list = [], []
                        for j in range(len(d['streams'])):
                            if "cubic" in d['streams'][j]['cc']:
                                cubic_data_seg_list.append(d['streams'][j]['data_segs'])
                            elif "bbr2" in d['streams'][j]['cc']:
                                bbr2_data_seg_list.append(d['streams'][j]['data_segs'])
                        
                        data_seg_sum_cubic.append( sum(cubic_data_seg_list) )
                        throughput_cubic = (sum(cubic_data_seg_list)*mss*8)/(end_time-start_time)/1e9
                        tput_cubic_p16.append( throughput_cubic )

                        data_seg_sum_bbr2.append( sum(bbr2_data_seg_list) )
                        throughput_bbr2 = (sum(bbr2_data_seg_list)*mss*8)/(end_time-start_time)/1e9
                        tput_bbr2_p16.append( throughput_bbr2 )

                except Exception as e:
                    print(e)

        except Exception as e:
            print(e)

    print("Throughput (P16)")
    print(f"BBRv2 - Mean: {np.mean(tput_bbr2_p16):.5f}  |  Coef. of Variance: {(np.std(tput_bbr2_p16)/np.mean(tput_bbr2_p16)):.5f}")
    print(f"CUBIC - Mean: {np.mean(tput_cubic_p16):.5f}  |  Coef. of Variance: {(np.std(tput_cubic_p16)/np.mean(tput_cubic_p16)):.5f}")
    # print(f"BBRv2 - Mean: {np.mean(tput_bbr2_p16):.5f}  |  Std. Dev.: {np.std(tput_bbr2_p16):.5f}  |  Coef. of Variance: {(np.std(tput_bbr2_p16)/np.mean(tput_bbr2_p16)):.5f}  |  Variance: {np.var(tput_bbr2_p16):.5f}")
    # print(f"CUBIC - Mean: {np.mean(tput_cubic_p16):.5f}  |  Std. Dev.: {np.std(tput_cubic_p16):.5f}  |  Coef. of Variance: {(np.std(tput_cubic_p16)/np.mean(tput_cubic_p16)):.5f}  |  Variance: {np.var(tput_cubic_p16):.5f}")

    # print("Data Segment (P16)")
    # print(f"BBRv2 - Mean: {np.mean(data_seg_sum_bbr2):.5f}  |  Std. Dev.: {np.std(data_seg_sum_bbr2):.5f}  |  Coef. of Variance: {(np.std(data_seg_sum_bbr2)/np.mean(data_seg_sum_bbr2)):.5f}  |  Variance: {np.var(data_seg_sum_bbr2):.5f}")
    # print(f"CUBIC - Mean: {np.mean(data_seg_sum_cubic):.5f}  |  Std. Dev.: {np.std(data_seg_sum_cubic):.5f}  |  Coef. of Variance: {(np.std(data_seg_sum_cubic)/np.mean(data_seg_sum_cubic)):.5f}  |  Variance: {np.var(data_seg_sum_cubic):.5f}")
    print(50*"=")

In [4]:
# import os
# import json
# import numpy as np
# from pathlib import Path

# rootdir = "/Users/eashan22/Desktop/Internship 2021/bbrv2/Brian's Project/experiment2"
# # rootdir = "/home/eadhikarla/Desktop/Brian's Project/bost-dtn"

# def traverse(path):
#     fileList = []
#     c = 0
#     for root, directories, files in os.walk(path):
#         for file in files:
#             if file.endswith("json") and float(file.split(":")[3].split(".json")[0].split("ms")[0])<30: # for filess where RTT is less than 30 
#                 fileList.append(os.path.join(root,file))
#                 c+=1
#     print(f"Total files: {c}")
#     # return fileList

# p = os.path.join(rootdir, "no-loss/pscheduler_bbr2_p1")
# traverse(p)

In [4]:
# paths = [
#         ("0.1-loss.v1/pscheduler_bbr2_p1",
#         "0.001-loss/pscheduler_bbr2_p16",
#         "0.1-loss.v1/pscheduler_bbr2_p16",
#         "0.01-loss.v1/pscheduler_bbr2_p1",
#         "0.01-loss.v1/pscheduler_bbr2_p16",
#         "no-loss/pscheduler_bbr2_p16"),

#         ("0.1-loss/pscheduler_cubic_p16",
#         "0.1-loss.v1/pscheduler_cubic_p1",   
#         "0.1-loss.v1/pscheduler_cubic_p16",
#         "0.001-loss/pscheduler_cubic_p16",
#         "0.01-loss.v1/pscheduler_cubic_p1",  
#         "0.01-loss.v1/pscheduler_cubic_p16",
#         "no-loss/pscheduler_cubic_p16"),

#         ("0.001-loss/pscheduler_both_p16",       
#         "0.1-loss.v1/pscheduler_both_p1",       
#         "0.1-loss.v1/pscheduler_both_p16",      
#         "0.01-loss.v1/pscheduler_both_p1",      
#         "0.01-loss.v1/pscheduler_both_p16",     
#         "0.1-loss.v2/pscheduler_both_p16",      
#         "0.1-loss.v2/pscheduler_both_p16_bbr1", 
#         "no-loss/pscheduler_both_p16")
#         ]


# # for p in paths:
# for q1 in paths[0]:
#     print(f"============================ {q1} ============================")
#     pscheduler_bbr2  = os.path.join(rootdir, q1)
#     filenames = traverse(pscheduler_bbr2)

#     data_seg = []
#     tput_bbr2_p1 = []
#     key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

#     for i,f in enumerate(filenames):
#         try:
#             path = Path(f)
#             data = [json.loads(line) for line in open(f, 'r')]
            
#             bbr2_data_seg, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0
#             for j,d in enumerate(data):
#                 if "interval" in d.keys() and j==0:
#                     start_time = data[j]['interval']['time'] # Start time of the test
#                 elif key2 in data[j].keys():
#                     end_time = data[j-1]['interval']['time'] # End time of the test
#                     mss = data[j-1]['interval']['mss'] # maximum segment size

#                     bbr2_data_seg = data[j][key2]
#                     data_seg.append( bbr2_data_seg )

#                     throughput = (bbr2_data_seg*mss*8)/(end_time-start_time)/1e9
#                     tput_bbr2_p1.append( throughput )
        
#         except Exception as e:
#             print(e)

#     print("Throughput (P1)")
#     print(f"BBRv2 - Mean: {np.mean(tput_bbr2_p1):.5f}  |  Std. Dev.: {np.std(tput_bbr2_p1):.5f}  |  Coef. of Variance: {(np.std(tput_bbr2_p1)/np.mean(tput_bbr2_p1)):.5f}  |  Variance: {np.var(tput_bbr2_p1):.5f}")
#     print("Data Segment (P1)")
#     print(f"BBRv2 - Mean: {np.mean(data_seg):.5f}  |  Std. Dev.: {np.std(data_seg):.5f}  |  Coef. of Variance: {(np.std(data_seg)/np.mean(data_seg)):.5f}  |  Variance: {np.var(data_seg):.5f}")
#     print("")

# for q2 in paths[1]:
#     print(f"============================ {q2} ============================")
#     pscheduler_cubic = os.path.join(rootdir, q2)
#     filenames = traverse(pscheduler_cubic)

#     data_seg = []
#     tput_p1_cubic = []
#     key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

#     for i,f in enumerate(filenames):
#         try:
#             path = Path(f)
#             # The MongoDB JSON dump has one object per line, so this works for me.
#             data = [json.loads(line) for line in open(f, 'r')]
            
#             cubic_data_seg, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0
#             for j,d in enumerate(data):
#                 if "interval" in d.keys() and j==0:
#                     start_time = data[j]['interval']['time'] # Start time of the test
#                 elif key1 in data[j].keys():
#                     end_time = data[j-1]['interval']['time'] # End time of the test
#                     mss = data[j-1]['interval']['mss'] # maximum segment size

#                     cubic_data_seg = data[j][key1]
#                     data_seg.append( cubic_data_seg )

#                     throughput = (cubic_data_seg*mss*8)/(end_time-start_time)/1e9
#                     tput_p1_cubic.append( throughput )
        
#         except Exception as e:
#             print(e)

#     print("Throughput (P1)")
#     print(f"CUBIC - Mean: {np.mean(tput_p1_cubic):.5f}  |  Std. Dev.: {np.std(tput_p1_cubic):.5f}  |  Coef. of Variance: {(np.std(tput_p1_cubic)/np.mean(tput_p1_cubic)):.5f}  |  Variance: {np.var(tput_p1_cubic):.5f}")
#     print("Data Segment (P1)")
#     print(f"CUBIC - Mean: {np.mean(data_seg):.5f}  |  Std. Dev.: {np.std(data_seg):.5f}  |  Coef. of Variance: {(np.std(data_seg)/np.mean(data_seg)):.5f}  |  Variance: {np.var(data_seg):.5f}")
#     print("")

# for q3 in paths[2]:
#     print(f"============================ {q3} ============================")
#     pscheduler_both = os.path.join(rootdir, q3)
#     filenames = traverse(pscheduler_both)

#     data_seg_sum_cubic, data_seg_sum_bbr2 = [], []
#     tput_cubic_p16, tput_bbr2_p16 = [], []
#     key1, key2, key3 = 'cubic_data_segs', 'bbr2_data_segs', 'bbr_data_segs'

#     for i,f in enumerate(filenames):
#         try:
#             path = Path(f)
#             data = [json.loads(line) for line in open(f, 'r')]
            
#             throughput_cubic, throughput_bbr2, throughput, mss, start_time, end_time = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0
#             for j,d in zip(range(len(data)),data):
#                 try:
#                     if "interval" in d.keys() and j==0:
#                         start_time = d['interval']['time'] # Start time of the test
#                     if "streams" in d.keys():
#                         end_time = data[j-1]['interval']['time'] # End time of the test
#                         mss = data[j-1]['interval']['mss'] # maximum segment size
                        
#                         cubic_data_seg_list, bbr2_data_seg_list = [], []
#                         for j in range(len(d['streams'])):
#                             if "cubic" in d['streams'][j]['cc']:
#                                 cubic_data_seg_list.append(d['streams'][j]['data_segs'])
#                             elif "bbr2" in d['streams'][j]['cc']:
#                                 bbr2_data_seg_list.append(d['streams'][j]['data_segs'])
                        
#                         data_seg_sum_cubic.append( sum(cubic_data_seg_list) )
#                         throughput_cubic = (sum(cubic_data_seg_list)*mss*8)/(end_time-start_time)/1e9
#                         tput_cubic_p16.append( throughput_cubic )

#                         data_seg_sum_bbr2.append( sum(bbr2_data_seg_list) )
#                         throughput_bbr2 = (sum(bbr2_data_seg_list)*mss*8)/(end_time-start_time)/1e9
#                         tput_bbr2_p16.append( throughput_bbr2 )

#                 except Exception as e:
#                     print(e)

#         except Exception as e:
#             print(e)

#     print("Throughput (P16)")
#     print(f"BBRv2 - Mean: {np.mean(tput_bbr2_p16):.5f}  |  Std. Dev.: {np.std(tput_bbr2_p16):.5f}  |  Coef. of Variance: {(np.std(tput_bbr2_p16)/np.mean(tput_bbr2_p16)):.5f}  |  Variance: {np.var(tput_bbr2_p16):.5f}")
#     print(f"CUBIC - Mean: {np.mean(tput_cubic_p16):.5f}  |  Std. Dev.: {np.std(tput_cubic_p16):.5f}  |  Coef. of Variance: {(np.std(tput_cubic_p16)/np.mean(tput_cubic_p16)):.5f}  |  Variance: {np.var(tput_cubic_p16):.5f}")

#     print("Data Segment (P16)")
#     print(f"BBRv2 - Mean: {np.mean(data_seg_sum_bbr2):.5f}  |  Std. Dev.: {np.std(data_seg_sum_bbr2):.5f}  |  Coef. of Variance: {(np.std(data_seg_sum_bbr2)/np.mean(data_seg_sum_bbr2)):.5f}  |  Variance: {np.var(data_seg_sum_bbr2):.5f}")
#     print(f"CUBIC - Mean: {np.mean(data_seg_sum_cubic):.5f}  |  Std. Dev.: {np.std(data_seg_sum_cubic):.5f}  |  Coef. of Variance: {(np.std(data_seg_sum_cubic)/np.mean(data_seg_sum_cubic)):.5f}  |  Variance: {np.var(data_seg_sum_cubic):.5f}")
#     print(50*"=")

============================ 0.1-loss.v1/pscheduler_bbr2_p1 ============================
Total files: 35
Throughput (P1)
BBRv2 - Mean: 2.22921  |  Std. Dev.: 0.03997  |  Coef. of Variance: 0.01793  |  Variance: 0.00160
Data Segment (P1)
BBRv2 - Mean: 9351449.48571  |  Std. Dev.: 173773.47248  |  Coef. of Variance: 0.01858  |  Variance: 30197219736.82122

============================ 0.001-loss/pscheduler_bbr2_p16 ============================
Total files: 70
Throughput (P1)
BBRv2 - Mean: 9.79684  |  Std. Dev.: 0.05101  |  Coef. of Variance: 0.00521  |  Variance: 0.00260
Data Segment (P1)
BBRv2 - Mean: 41181799.81429  |  Std. Dev.: 229777.31653  |  Coef. of Variance: 0.00558  |  Variance: 52797615192.86552

============================ 0.1-loss.v1/pscheduler_bbr2_p16 ============================
Total files: 35
Throughput (P1)
BBRv2 - Mean: 9.80138  |  Std. Dev.: 0.03258  |  Coef. of Variance: 0.00332  |  Variance: 0.00106
Data Segment (P1)
BBRv2 - Mean: 41214670.34286  |  Std. Dev.: 134

<ipython-input-4-7b9ef5707892>:149: RuntimeWarning: invalid value encountered in double_scalars
  print(f"BBRv2 - Mean: {np.mean(tput_bbr2_p16):.5f}  |  Std. Dev.: {np.std(tput_bbr2_p16):.5f}  |  Coef. of Variance: {(np.std(tput_bbr2_p16)/np.mean(tput_bbr2_p16)):.5f}  |  Variance: {np.var(tput_bbr2_p16):.5f}")
<ipython-input-4-7b9ef5707892>:153: RuntimeWarning: invalid value encountered in double_scalars
  print(f"BBRv2 - Mean: {np.mean(data_seg_sum_bbr2):.5f}  |  Std. Dev.: {np.std(data_seg_sum_bbr2):.5f}  |  Coef. of Variance: {(np.std(data_seg_sum_bbr2)/np.mean(data_seg_sum_bbr2)):.5f}  |  Variance: {np.var(data_seg_sum_bbr2):.5f}")


Throughput (P16)
BBRv2 - Mean: 8.57664  |  Std. Dev.: 1.13554  |  Coef. of Variance: 0.13240  |  Variance: 1.28945
CUBIC - Mean: 1.22878  |  Std. Dev.: 1.16579  |  Coef. of Variance: 0.94874  |  Variance: 1.35908
Data Segment (P16)
BBRv2 - Mean: 36063661.57143  |  Std. Dev.: 4835482.92428  |  Coef. of Variance: 0.13408  |  Variance: 23381895111017.26953
CUBIC - Mean: 5156028.65714  |  Std. Dev.: 4883125.86587  |  Coef. of Variance: 0.94707  |  Variance: 23844918221903.59375
============================ 0.01-loss.v1/pscheduler_both_p1 ============================
Total files: 20
Throughput (P16)
BBRv2 - Mean: 0.00000  |  Std. Dev.: 0.00000  |  Coef. of Variance: nan  |  Variance: 0.00000
CUBIC - Mean: 0.86197  |  Std. Dev.: 0.61398  |  Coef. of Variance: 0.71229  |  Variance: 0.37697
Data Segment (P16)
BBRv2 - Mean: 0.00000  |  Std. Dev.: 0.00000  |  Coef. of Variance: nan  |  Variance: 0.00000
CUBIC - Mean: 3612716.70000  |  Std. Dev.: 2572222.77993  |  Coef. of Variance: 0.71199  |  V